In [1]:
import time
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    
opciones.add_argument('--start-maximized')         
opciones.add_argument('--incognito')
opciones.add_argument("--disable-cookies")


In [2]:
# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opciones)

url = "https://www.booking.com/searchresults.pt-br.html?ss=Perdizes%2C+S%C3%A3o+Paulo%2C+Estado+de+S%C3%A3o+Paulo%2C+Brasil&ssne=Jardins&ssne_untouched=Jardins&label=pt-br-booking-desktop-9_uvqir24qvA6x6xGiDvCQS652796015463%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9061033%3Ali1001773%3Adec%3Adm&aid=2311236&lang=pt-br&sb=1&src_elem=sb&src=searchresults&dest_id=2352&dest_type=district&ac_position=0&ac_click_type=b&ac_langcode=xb&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=797d7f61fea2020d&ac_meta=GhA3OTdkN2Y2MWZlYTIwMjBkIAAoATICeGI6B3BlcmRpemVAAEoAUAA%3D&checkin=2023-09-12&checkout=2023-09-13&group_adults=2&no_rooms=1&group_children=0"
driver.get(url)

In [3]:
wait = WebDriverWait(driver, 20)

try:
    cookie_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler")))
    cookie_button.click()
    
    
except TimeoutException:
    pass  

In [4]:
button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="header-currency-picker-trigger"]')
button.click()

In [5]:
button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="selection-item"]')
button.click()

In [6]:
try:
    
    button_element = driver.find_element(By.XPATH, "//button[@aria-label='Ignorar informações de login.']")
    button_element.click()
except NoSuchElementException:
    pass

In [14]:
properties = []


def extract_properties():
    property_cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')
    for property in property_cards:
        name = property.find_element(By.CSS_SELECTOR, 'div[data-testid="title"]').text
        price = property.find_element(By.CSS_SELECTOR, '[data-testid="price-and-discounted-price"]').text
        chk_out = property.find_element(By.XPATH, '//button[contains(@class, "d47738b911") and @data-testid="date-display-field-end"]').text
        
        new_property = {}
        new_property['name'] = name
        new_property['price'] = price
        new_property['chk_out'] = chk_out
        
        properties.append(new_property)


total_pages = int(driver.find_element(By.CSS_SELECTOR, 'div[data-testid="pagination"] li:last-child').text)
print(f"Total pages: {total_pages}")

current_page = 0

Total pages: 2


In [15]:
for current_page in range(total_pages):
    driver.delete_all_cookies()
    extract_properties()
    next_page_btn = driver.find_element(By.CSS_SELECTOR, 'button[aria-label*="Página seguinte"]')
    next_page_btn.click()

In [16]:
print(properties)

[{'name': 'Share em Perdizes com ótima área comum', 'price': 'R$ 362', 'chk_out': 'qua., 13 de set.'}, {'name': 'Belíssimo Stúdio', 'price': 'R$ 300', 'chk_out': 'qua., 13 de set.'}, {'name': 'Hospedaria Santo André', 'price': 'R$ 288', 'chk_out': 'qua., 13 de set.'}, {'name': 'FLAT PERDIZES - Executivo', 'price': 'R$ 459', 'chk_out': 'qua., 13 de set.'}, {'name': 'Housi Perdizes By Charlie', 'price': 'R$ 360', 'chk_out': 'qua., 13 de set.'}, {'name': 'Apartamento Palmeiras', 'price': 'R$ 411', 'chk_out': 'qua., 13 de set.'}, {'name': 'Studio Perdizes Allianz', 'price': 'R$ 350', 'chk_out': 'qua., 13 de set.'}, {'name': 'Prédio NOVO em frente ao Majestoso Allianz Parque', 'price': 'R$ 375', 'chk_out': 'qua., 13 de set.'}, {'name': 'Noon Vila Madalena', 'price': 'R$ 544', 'chk_out': 'qua., 13 de set.'}, {'name': 'Vossa Bossa Allianz', 'price': 'R$ 355', 'chk_out': 'qua., 13 de set.'}, {'name': 'Mondo Verde Hostel', 'price': 'R$ 117', 'chk_out': 'qua., 13 de set.'}, {'name': 'Estúdio a 4

In [17]:
import pandas as pd
df = pd.DataFrame(properties)
df.head()

,name,price,chk_out
0,Share em Perdizes com ótima área comum,R$ 362,"qua., 13 de set."
1,Belíssimo Stúdio,R$ 300,"qua., 13 de set."
2,Hospedaria Santo André,R$ 288,"qua., 13 de set."
3,FLAT PERDIZES - Executivo,R$ 459,"qua., 13 de set."
4,Housi Perdizes By Charlie,R$ 360,"qua., 13 de set."


In [18]:
df.shape

(38, 3)

In [19]:
df.info

<bound method DataFrame.info of                                                  name     price  \
0              Share em Perdizes com ótima área comum    R$ 362   
1                                    Belíssimo Stúdio    R$ 300   
2                              Hospedaria Santo André    R$ 288   
3                           FLAT PERDIZES - Executivo    R$ 459   
4                           Housi Perdizes By Charlie    R$ 360   
5                               Apartamento Palmeiras    R$ 411   
6                             Studio Perdizes Allianz    R$ 350   
7   Prédio NOVO em frente ao Majestoso Allianz Parque    R$ 375   
8                                  Noon Vila Madalena    R$ 544   
9                                 Vossa Bossa Allianz    R$ 355   
10                                 Mondo Verde Hostel    R$ 117   
11             Estúdio a 400 metros do Allianz Parque    R$ 448   
12  Studios Novíssimos com Piscina no Rooftop e vi...    R$ 329   
13      360 Suítes Perdizes - 

In [20]:
name_counts = df['name'].value_counts()
print(name_counts)

name
Casa SP011                                                                                                  2
Share em Perdizes com ótima área comum                                                                      1
Apartamento novo e luxuoso ao lado do Allianz                                                               1
VS12 Apartamento P4 Pessoas Lindo e Moderno em Perdizes Prox Allianz Com Vaga Garagem                       1
Allianz Parque Studio em São Paulo                                                                          1
Studio Allianz Perdizes                                                                                     1
Vila Rock Hostel - próximo Allianz Parque, Vila Madalena, Av Paulista, Hospital das Clínicas INCOR FMUSP    1
Mimos Hospedagem                                                                                            1
STUDIO - VN Apiacas próximo ao Allianz Parque á 750 metros - Perdizes                                       1
Studi

In [21]:
df.to_csv('perdizes.csv', index=False)

In [22]:
driver.quit()